In [1]:
import pandas as pd
import myslack
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
import pickle
import time
import re
from tqdm import tqdm
from tqdm import trange

import requests
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
driver = webdriver.Chrome()  
url = 'https://www.yogiyo.co.kr/mobile/?utm_source=google&utm_medium=cpc&utm_campaign=sem_bra_orignal&utm_term=%EC%9A%94%EA%B8%B0%EC%9A%94&utm_id=sem_000001&referrer=adjust_tracker%3Dgrm2aa%26adjust_google_network%3Dg%26adjust_google_placement%3D%26adjust_campaign%3Dsem_bra_web_kr_700886066_cpc%26adjust_adgroup%3D38249737818%26adjust_creative%3D%EC%9A%94%EA%B8%B0%EC%9A%94_b&gclid=EAIaIQobChMIv9DaxIL-3AIVA6yWCh0C8glmEAAYASAAEgLbF_D_BwE#/%EC%84%9C%EC%9A%B8/138223/'
driver.get(url) 

In [3]:
food_dict = {
    '프랜차이즈':3, '치킨':4, '피자':5, '양식':5, '중국집':6,
    '한식':7, '일식':8, '돈가스':8, '족발':9, '보쌈':9, 
    '야식':10, '분식':11, '카페':12, '디저트':12 }

In [4]:
# 0. 페이지 한번 맨아래로 내리기
def scroll_bottom():
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

# 1. 입력한 위치로 설정하기
def set_location(location):
    print(location+'으로 위치 설정 하는중...')
    driver.find_element_by_css_selector('#search > div > form > input').click()
    driver.find_element_by_css_selector('#button_search_address > button.btn-search-location-cancel.btn-search-location.btn.btn-default > span').click()
    driver.find_element_by_css_selector('#search > div > form > input').send_keys(location)
    driver.find_element_by_css_selector('#button_search_address > button.btn.btn-default.ico-pick').click()
    time.sleep(2)
    print(location+'으로 위치 설정 완료!')
    
# 2. 카테고리 페이지로 넘어가기
def go_to_category(category):
    print(category+' 카테고리 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="category"]/ul/li[{}]/span'.format(food_dict.get(category))).click()
    time.sleep(2)
    print(category+' 카테고리 페이지 로드 완료!')    

# 3. 카테고리(음식점 리스트) 페이지 모두 펼치기
def stretch_list_page():
    restaurant_count = int(driver.find_element_by_css_selector('#restaurant_count').text)
    scroll_count = int((restaurant_count/20))
    print('모든 음식점 리스트 불러오기 시작...')
    for _ in trange(scroll_count):
        try:
            scroll_bottom()
            time.sleep(3)
        except Exception as e:
            pass
    scroll_bottom()
    print('모든 음식점 리스트 불러오기 완료!')
    
# 4. 해당 카테고리 음식점 리스트 리턴
def get_restaurant_list():
    restaurant_list=[]
    restaurants = driver.find_elements_by_css_selector('#content > div > div.restaurant-list > div.col-sm-6.contract')
    for restaurant in restaurants:
        restaurant_list.append(restaurant.find_element_by_css_selector('div > table > tbody > tr > td:nth-child(2) > div > div.restaurant-name.ng-binding').text)
    return list(set(restaurant_list))

# 5. 검색창에 음식점 검색하기
def search_restaurant(restaurant_name):
    try:
        driver.find_element_by_xpath('//*[@id="category"]/ul/li[1]/a').click()
        driver.find_element_by_xpath('//*[@id="category"]/ul/li[13]/form/div/input').send_keys(restaurant_name)    
        driver.find_element_by_xpath('//*[@id="category_search_button"]').click()
    except Exception as e:
        print('search_restaurant 에러')
    time.sleep(2)

# 6. 검색한 음식점 클릭
def go_to_restaurant():
    try:
        driver.find_element_by_xpath('//*[@id="content"]/div/div[4]/div[2]/div').click()
    except Exception as e:
        print('go_to_restaurant 에러')
    time.sleep(2)

# 7. 해당 음식점의 리뷰 페이지로 넘어가기
def go_to_review():
    print('리뷰 페이지 로드중...')
    driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a').click()
    time.sleep(2)
    print('리뷰 페이지 로드 완료!')
    
# 8. 리뷰 더보기 클릭하기 
def click_more_review():
    driver.find_element_by_class_name('btn-more').click()
    time.sleep(2)
    
# 9. 리뷰 페이지 모두 펼치기
def stretch_review_page():
    review_count = int(driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text)
    click_count = int((review_count/10))
    print('모든 리뷰 불러오기 시작...')
    for _ in trange(click_count):
        try:
            scroll_bottom()
            click_more_review()
        except Exception as e:
            pass
    scroll_bottom()
    print('모든 리뷰 불러오기 완료!')
        
# 10. 해당 음식점의 모든 리뷰 객체 리턴
def get_all_review_elements():
    reviews = driver.find_elements_by_css_selector('#review > li.list-group-item.star-point.ng-scope')
    return reviews
        
# 11. 페이지 뒤로 가기 (한 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감)
def go_back_page():
    print('페이지 돌아가기중...')
    driver.execute_script("window.history.go(-1)")
    time.sleep(3)
    print('페이지 돌아가기 완료!')

---

In [5]:
def yogiyo_crawling(location, category):
    df = pd.DataFrame(columns=['Restaurant','UserID','Menu','Review',
                                   'Total','Taste','Quantity','Delivery','Date'])
    
    try:
        set_location(location) # 검색할 지역 설정
        go_to_category(category) # 해당 카테고리(음식점 리스트) 페이지로 넘어감

        print('Start {} - {} Crawling...'.format(location, category))
        
        stretch_list_page() # 카테고리(음식점 리스트) 페이지 모두 펼치기
        restaurant_list = get_restaurant_list() # 해당 카테고리 음식점 리스트 받아오기
        
        for restaurant_name in restaurant_list:
            try:
                print('********** '+restaurant_name+' **********')
                search_restaurant(restaurant_name) # 음식점을 순서대로 검색
                go_to_restaurant() # 검색한 음식점 클릭             
                go_to_review() # 해당 음식점의 리뷰페이지로 넘어감
                stretch_review_page() # 해당 음식점의 모든 리뷰를 불러옴

                for review in tqdm(get_all_review_elements()):  # 해당 음식점의 리뷰 수 만큼 데이터를 가져옴
                    try:
                        df.loc[len(df)] = { 
                            'Restaurant':driver.find_element_by_class_name('restaurant-name').text,
                            'UserID':review.find_element_by_css_selector('span.review-id.ng-binding').text,
                            'Menu':review.find_element_by_css_selector('div.order-items.default.ng-binding').text,
                            'Review':review.find_element_by_css_selector('p').text,
                            'Total':str(len(review.find_elements_by_css_selector('div > span.total > span.full.ng-scope'))),
                            'Taste':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(3)').text,
                            'Quantity':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(6)').text,
                            'Delivery':review.find_element_by_css_selector('div:nth-child(2) > div > span.category > span:nth-child(9)').text,
                            'Date':review.find_element_by_css_selector('div:nth-child(1) > span.review-time.ng-binding').text,
                        }
                    except Exception as e:
                        print('리뷰 페이지 에러')
                        print(e)
                        pass
                    
            except Exception as e:
                print('*** '+restaurant_name+' *** 음식점 페이지 에러')
                go_back_page()
                print(e)
                pass
            
            print('음식점 리스트 페이지로 돌아가는중...')
            go_back_page() # 해당 음식점 리뷰를 모두 모았으면 다시 음식점 리스트 페이지로 돌아감
            time.sleep(2)
            
    except Exception as e:
        print('음식점 리스트 페이지 에러')
        print(e)
        pass
        
    print('End of {} {} Crawling!'.format(location, category))
    myslack.send_slack('{} {} crawling finish!!!'.format(location, category))
    return df

---

In [6]:
%%time 

location='역삼동'
category='치킨'

yogiyo = yogiyo_crawling(location, category)

역삼동으로 위치 설정 하는중...
역삼동으로 위치 설정 완료!
치킨 카테고리 페이지 로드중...


  0%|          | 0/4 [00:00<?, ?it/s]

치킨 카테고리 페이지 로드 완료!
Start 역삼동 - 치킨 Crawling...
모든 음식점 리스트 불러오기 시작...


100%|██████████| 4/4 [00:12<00:00,  3.01s/it]


모든 음식점 리스트 불러오기 완료!
********** 멕시카나-선릉역점 **********
리뷰 페이지 로드중...


...

리뷰 페이지 로드 완료!
모든 리뷰 불러오기 시작...


  0%|          | 0/148 [00:00<?, ?it/s]

모든 리뷰 불러오기 완료!


100%|██████████| 148/148 [00:43<00:00,  3.40it/s]


음식점 리스트 페이지로 돌아가는중...
페이지 돌아가기중...
페이지 돌아가기 완료!
End of 역삼동 치킨 Crawling!
<Response [200]>
CPU times: user 2min 3s, sys: 7.92 s, total: 2min 10s
Wall time: 3h 47min 5s


In [7]:
yogiyo.tail()

NameError: name 'chicken_df' is not defined

In [8]:
pickle.dump(yogiyo, open('chicken_df_{}_{}.pkl'.format(location, category),'wb'))

In [13]:
yogiyo_df = pickle.load(open('chicken_df_{}_{}.pkl'.format(location, category),'rb'))

In [14]:
yogiyo_df.tail()

,Restaurant,UserID,Menu,Review,Total,Taste,Quantity,Delivery,Date
9927,강남929치킨피자,dp**님,크리스피 순살/1,Delcious! And very impressed the delivery driv...,5,5,5,5,2018년 4월 28일
9928,강남929치킨피자,pe**님,세트6（슈퍼슈프림＋크리스피후라이드）/1(메뉴 변경 선택(변경 안함)),진짜 맛있네요 치킨 바삭하고 간 적당하고 느끼하지도 않고 피자도 딱히 흠잡을 곳이 ...,5,5,5,5,2018년 4월 27일
9929,강남929치킨피자,da**님,슈퍼슈프림피자/1,피자 시켰는데 배달 예정시간보다 일찍 왔고 일단 피자가 그림에서처럼 팬피자는 아니고...,4,5,4,5,2018년 4월 25일
9930,강남929치킨피자,wk**님,슈퍼슈프림피자/1,너무 배고파서 피자를 주문! 강남929 란 처음보는 피자배달 배달도 빨리오고 가격도...,5,5,5,5,2018년 4월 25일
9931,강남929치킨피자,ci**님,크리스피후라이드/1,퇴근 후 먹는 치킨 배달이 이렇게 빠를 줄이야..요즘 배달 다들 늦는데 이렇게빠르게...,5,5,5,5,2018년 4월 25일


In [16]:
yogiyo_df['Restaurant'].value_counts()

굽네치킨-역삼2동점         2121
후아바베큐-논현직영점        1300
강정이기가막혀-역삼점        1205
BBQ-역삼스타점           676
롯데리아-역삼점            596
지코바치킨               416
델리치킨                414
멕시카나-선릉역점           401
어니언치킨-역삼점           382
깻잎두마리칩킨-역삼점         252
후라이드참잘하는집-역삼점       232
양동시장통닭-언주역본점        207
가성비닭                203
치킨플러스-대치점           176
네네치킨-서초중앙점          153
네네치킨-역삼충현점          152
페리카나-대치동점           150
강남929치킨피자           148
502세컨즈카페-강남점        133
브라보치킨피자-역삼점         132
쌀쌀맞은닭-역삼점           107
노랑통닭-역삼점             94
미니닭날개구이              50
페리카나-논현점             50
삼풍치킨-역삼점             31
호또치킨 본점              30
똥깨네옛날통닭              30
누구나홀닭반한닭-역삼개나리점      27
체르치킨-논현점             17
미스터존슨바베큐-강남점         15
컬투치킨-역삼사랑점           11
미담숯불바베큐-본점            6
국보치킨                  5
                      4
삼총사                   3
코코참숯바베큐-강남점           2
펀비어킹-경복사거리점           1
Name: Restaurant, dtype: int64

In [129]:
driver.close() # 창닫기

In [141]:
driver.quit() # 브라우져 닫기

---